In [ ]:
#gui
#webcraping from cricbuzz to get all score
#display the live score update on python app

In [5]:
pip install Pillow

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install bs4

  Using cached bs4-0.0.1.tar.gz (1.1 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1264 sha256=973c326633b36f8d7fab04f1d555c9cb7128709489676f91a9c97e4f66b50660
  Stored in directory: c:\users\soham\appdata\local\pip\cache\wheels\d4\c8\5b\b5be9c20e5e4503d04a6eac8a3cd5c2393505c29f02bea0960
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [1]:

from tkinter import *
from tkinter import Tk
from tkinter.ttk import Combobox
from PIL import ImageTk 
from bs4 import BeautifulSoup
import requests
import re


In [31]:
class CricketScore:
    #Constructor
    def __init__(self, rootWindow):
        self.rootWindow = rootWindow
        self.rootWindow.title("LIVE CRICKET SCORE")
        self.rootWindow.geometry('800x700')
        self.bg = ImageTk.PhotoImage(file="cric.jpg")
        bg = Label(self.rootWindow, image=self.bg).place(x=0, y=0)

        #give lable to gui
        self.label = Label(self.rootWindow,text='Live Match',font=('impact',60),compound='center').pack(padx=100,pady=50)
        #Featch the live match detalil and set it in gui
        self.var = StringVar()#here we are dumping data
        self.matches =self.match_detail()#here we declare the funton so we need to define that
        self.data = [i for i in self.matches.keys()]
        self.cb = Combobox(self.rootWindow,values= self.data, width=60)
        self.cb.place(x=210,y=190)

        #button to check the details
        self.b1=Button(self.rootWindow,text="Check Score",font=("impact",20),command=self.show_match_details).place(x=20,y=600)
    
    
    #creation Command for check socre button
    def select(self):
        return self.cb.get()

    
    #funciton to scrap the website cricbazz
    def scrap(self):
        URL = "https://www.cricbuzz.com/"
        page=requests.get(URL)#here we make the http request
        #parse the resposnse and fetch the data
        soup = BeautifulSoup(page.content,"html.parser")#here we get all html data
        results = soup.find(id="match_menu_container") #finding the right div
        scrap_result= results.find_all("li",class_="cb-match-card")#cb match card show proper match cards 
        return scrap_result

    
    #we will lode the all matches details 
    def match_detail(self):
        #scrap the data
        details = self.scrap()#here we declare the funton so we need to define that
                                #by scrap funtion details get all score card

        #need to prese & read the relevent data
        live_match={}
        for detail in details:
            live_team_details={}
            summary = self.match_summary(detail)
            if summary is not None:
                match_header= self.match_header(detail).text
                teams= self.teams_name(detail)
                score_card= self.team_score(detail)

                live_team_details['summary']=summary.text
                live_team_details['match_header']=match_header
                live_team_details['score_card']=score_card[0]+ " :: "+ score_card[1]
                live_match[teams[0] + "vs" + teams[1]]=live_team_details
        return live_match
    
    
    #frunction to fetch the given match summary
    def match_summary(self,detail):
        return detail.find("div",class_="cb-mtch-crd-state")
        

    #fruntion of fetch the match details
    def match_header(self,detail):
        return detail.find("div",class_="cb-mtch-crd-hdr")

    
    #function to find the team name
    def teams_name(self,detail):
        t=[]
        team1_element = detail.find("div", class_="cb-hmscg-bat-txt")
        if team1_element:
            team1_details = team1_element.text
            team1_index = re.search(r"\d", team1_details).start() if re.search(r"\d", team1_details) else len(team1_details)
            t.append(team1_details[:team1_index])

        team2_element = detail.find("div", class_="cb-hmscg-bwl-txt")
        if team2_element:
            team2_details = team2_element.text
            team2_index = re.search(r"\d", team2_details).start() if re.search(r"\d", team2_details) else len(team2_details)
            t.append(team2_details[:team2_index])

        return t

    
    #function to featch the score of team 
    def team_score(self,detail):
        t=[]        
        team1_element = detail.find("div", class_="cb-hmscg-bat-txt")
        if team1_element:#it check value is none or not
            team1_details = team1_element.text
            t.append(team1_details)
        
        team2_element = detail.find("div", class_="cb-hmscg-bwl-txt")
        if team2_element:
            team2_details=team2_element.text
            t.append(team2_details)
        
        return t

    
    #function to show mathch details
    def show_match_details(self):
        #building match details frame
        self.frame1 = Frame(self.rootWindow, bg="#F39A01")
        self.frame1.place(x=100,y=240,width=600,height=300)

        #fetching details of the match
        x = self.matches[self.select()]

        #displaying team nemes
        Label(self.frame1, text=self.select() + " - " + x['match_header'], font=("impact", 20), bg="#F39A01", fg="#0047AB",bd=0).place(x=10, y=15)

        # Dispalying details of the match
        Label(self.frame1, text="Score Details : ", font=("impact", 20), bg="#F39A01", fg="black",bd=0).place(x=10, y=60)
        Label(self.frame1, text=x['score_card'], font=("impact", 20 ), bg="#F39A01", fg="black",bd=0).place(x=20, y=100)
        Label(self.frame1, text="Summary : ", font=("impact", 20 ), bg="#F39A01", fg="black",bd=0).place(x=10, y=180)
        Label(self.frame1, text=x['summary'], font=("impact", 20), bg="#F39A01", fg="black",bd=0).place(x=20, y=220)


#main funtion to start
def main():
    # creating tkinter window
    rootWindow = Tk()#object of tk
    
    # creating object for class cricket_score
    obj = CricketScore(rootWindow)
   
    # starting the gui
    rootWindow.mainloop()


if __name__=='__main__':
    main()